# Monte Carlo simulation to obtain global minimum

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import random
import math
from plotly.subplots import make_subplots
from ipywidgets import FloatSlider, IntSlider, FloatLogSlider, Button, Output, HBox, VBox, IntProgress

In [ ]:
A1 = 1. # potential curvature first axis
A2 = 1.e-1 # potential curvature second axis

# Starting point of our MC, for the two coordinates
starting_x1 = 9.0
starting_x2 = 9.0

def get_energy(x, y):
    return 0.5 * A1 * x**2 + 0.5 * A2 * (y**4 - 20 * y**2)

def get_property(x1, x2):
    # I can do something else, I decide to sample the energy of one of the two coordinates
    #return 0.5 * A2 * (x2**4 - 4 * x2**2)
    
    # I just return the second coordinate, to see its average
    return x2

In [ ]:
# fig1
x, y = np.mgrid[-10:10:100j, -10:10:100j]
z = get_energy(x, y)

fig1 = go.FigureWidget(data=[go.Surface(z=z, x=x, y=y, opacity=0.5),
                             go.Scatter3d(x=[starting_x1], 
                                          y=[starting_x2], 
                                          z=[get_energy(starting_x1, starting_x2)], 
                                          mode='markers+text',
                                          text=['Current position'],
                                          marker=dict(size=8, color='red'))])

In [ ]:
# fig2

fig2 = go.FigureWidget()

fig2.add_trace(go.Scatter(
    x=[starting_x1],
    y=[starting_x2],
    mode='lines+markers',
    visible = False,
    showlegend=False,
    marker=dict(
        symbol='x',
        opacity=0.7,
        color='white',
        size=8,
        line=dict(width=1),
    ),
))

fig2.add_trace(go.Histogram2d(
    x=[],
    y=[],
    histnorm='probability',
    autobinx = False,
    xbins=dict(start=-10, end=10, size=0.1),
    autobiny = False,
    ybins=dict(start=-10, end=10, size=0.1),
    colorscale=[[0, 'rgb(12,51,131)'], [0.25, 'rgb(10,136,186)'], [0.6, 'rgb(242,211,56)'], 
                [0.75, 'rgb(242,143,56)'], [1, 'rgb(217,30,30)']]
))

fig1.update_layout(title='Potential energy surface', autosize=False,
                  width=420, height=420,
                  margin=dict(l=10, r=10, b=30, t=30))

fig2.update_layout(
    xaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
    yaxis=dict( ticks='', showgrid=False, zeroline=False, nticks=20 ),
    xaxis_title="X",
    yaxis_title="Y",
    autosize=False,
    height=420,
    width=430,
    margin=dict(l=30, r=30, b=60, t=60),
    hovermode='closest',

)

fig3 = go.FigureWidget()

fig3.add_trace(go.Scatter(x=[], y=[], mode='lines',
    name='Total energy',
    line=dict(color='red', width=2),
    connectgaps=True,
))

fig3.update_layout(
    height = 200,
    width = 530,
    xaxis_title = 'Monte-Carlo move',
    yaxis_title = 'Total energy',
    margin=dict(l=30, r=30, b=10, t=10),
)

display(HBox([fig1, fig2]))

In [ ]:
def run_mc(move_size, num_iterations, temp):
    """
    :param move_size: max move size for the two axes x1 and x2.
        In principle, the move size could/should be different for x1 and x2, here I choose the same for simplicity!
        
    :param num_iterations: total number of Monte-Carlo iterations
    """
    x1 = starting_x1
    x2 = starting_x2
    energy = get_energy(x1, x2)

    all_x1 = [x1]
    all_x2 = [x2]
    all_properties = [get_property(x1, x2)]
    all_energies = [get_energy(x1, x2)]
    count_accepted = 0
    count_refused = 0

    for iter_cnt in range(num_iterations):
        # shift by a random value between -move_size and + move_size
        new_x1 = x1 + ((random.random() - 0.5) * 2) * move_size
        new_x2 = x2 + ((random.random() - 0.5) * 2) * move_size
        new_energy = get_energy(new_x1, new_x2)

        if new_energy < energy:
            accepted = True
        else:
            # boltzmann_k = 1 in these units
            probability = math.exp(-(new_energy - energy)/temp)
            # random.random() is a random number between 0 and 1
            # also probability is between 0 and 1
            # so if I accept only if random.random() is < probability,
            # I'm accepting the move with 'probability' probability
            accepted = random.random() < probability

        if accepted:
            x1 = new_x1
            x2 = new_x2
            energy = new_energy
            count_accepted += 1
        else:
            count_refused += 1
        all_properties.append(get_property(x1, x2))
        all_energies.append(energy)
        all_x1.append(x1)
        all_x2.append(x2)
        
    average_integrated_energy = np.cumsum(all_energies) / (1 + np.arange(len(all_energies)))
    average_property = np.cumsum(all_properties) / (1 + np.arange(len(all_properties)))

    return count_accepted, count_refused, average_integrated_energy, average_property, all_x1, all_x2

In [ ]:
temp_slider = FloatSlider(min=0.1, max=5, value=0.65, continuous_update=False, description="Temperature")
move_size_slider = FloatLogSlider(min=-1, max=1, value=0.25, continuous_update=False, description="Max move size")
num_iterations_slider = IntSlider(min=100000, max=1000000, value=100000, continuous_update=False, description="Num iterations")
run_button = Button(description="Run Monte-Carlo")
run_button.style.button_color = 'green'
traces_button = Button(description="Show Traces")
x_slider = FloatSlider(value=9.0, min=-10.0, max=10.0, continuous_update=False, description="Initial pos. x")
y_slider = FloatSlider(value=9.0, min=-10.0, max=10.0, continuous_update=False, description="Initial pos. y")

In [ ]:
controls = VBox([x_slider, y_slider, temp_slider, move_size_slider, num_iterations_slider, run_button, traces_button])

display(HBox([controls, fig3]))

In [ ]:
def change_init_position(c):
    starting_x1 = x_slider.value
    starting_x2 = y_slider.value
    fig1.data[1].x = [x_slider.value]
    fig1.data[1].y = [y_slider.value]
    fig1.data[1].z = [get_energy(x_slider.value, y_slider.value)]
    
x_slider.observe(change_init_position, names='value')
y_slider.observe(change_init_position, names='value')

In [ ]:
def interactive_plot(button):
    global all_x1, all_x2
    temp = temp_slider.value
    move_size=move_size_slider.value
    num_iterations=num_iterations_slider.value
    
    run_button.disabled = True
    run_button.style.button_color = 'red'
    run_button.description = "Runing..."
    traces_button.disabled = True
    try:
        count_accepted, count_refused, average_integrated_energy, average_property, all_x1, all_x2 = run_mc(
            move_size=move_size, num_iterations=num_iterations, temp=temp)
        
        fig1.data[1].x = [all_x1[-1]]
        fig1.data[1].y = [all_x2[-1]]
        fig1.data[1].z = [get_energy(all_x1[-1], all_x2[-1])]
    
        snum = int(len(all_x1)/100)
        fig2.data[0].x = all_x1[::snum]
        fig2.data[0].y = all_x2[::snum]
        
        fig2.data[1].x = all_x1
        fig2.data[1].y = all_x2
        
        skip_start = 10000
        
        fig3.data[0].x = np.arange(len(average_integrated_energy) - skip_start)
        fig3.data[0].y = average_integrated_energy[skip_start:]
    finally:
        run_button.disabled = False
        traces_button.disabled = False
        run_button.style.button_color = 'green'
        run_button.description="Run Monte-Carlo"
        
run_button.on_click(interactive_plot)

In [ ]:
def toggle_show_traces(button):
    fig2.data[0].visible ^= True
    if fig2.data[0].visible:
        traces_button.description = "Hide traces"
    else:
        traces_button.description = "Show traces"
    
traces_button.on_click(toggle_show_traces)


def show_current_point(trace, points, selector):
    for i in points.point_inds:
        fig1.data[1].x = [all_x1[i]]
        fig1.data[1].y = [all_x2[i]]
        fig1.data[1].z = [get_energy(all_x1[i], all_x2[i])]
        
fig3.data[0].on_click(show_current_point)